In [1]:
!pip install langchain langchain-openai openai tiktoken -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from getpass import getpass

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [3]:
# import os
# from langchain_openai import ChatOpenAI


# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # Инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇

In [4]:
# !wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [5]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

💾 Добавим памяти LLM'ке! 🦙







In [6]:
# подгрузим датасет
import pandas as pd

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1084404/dial_df.csv")

In [15]:
# Импортируйте модуль памяти
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

In [16]:
# создайте цепочку диалога, в параметры передайте языковую модель и память
template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
Answer with only one number.

Current conversation:

{chat_history}
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)

memory = ConversationBufferMemory(memory_key="chat_history")

conversation_buf_mem = LLMChain(
    prompt=prompt,
    llm=llm,  # языковая модель
    memory=memory, # модуль памяти
    verbose=True, # отображать логи в консоли или нет
)

/var/folders/1v/2njvtrl903s020_gk0tw647c0000gn/T/ipykernel_20617/1462234541.py:19: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation_buf_mem = LLMChain(


In [11]:
df

,dialogue_id,question
0,1,Под каким номером играл Криштиану Роналду в Юв...
1,1,Под каким номером он играл в Манчестер Юнайтед?
2,1,В каком году он был рожден?
3,2,Сколько лет прожил величайший русский поэт Але...
4,2,В каком году он родился?
5,2,Дуэль в каком году стала для него последней?
6,3,Сколько было участников в золотом составе груп...
7,3,В каком году родился барабанщик золотого соста...
8,3,В каком году распалась группа?
9,4,В каком году родился Альберт Эйнштейн?


In [30]:
conversation_buf_mem.predict(human_input=df['question'].iloc[1])



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
Answer with only one number.

Current conversation:

Human: Под каким номером играл Криштиану Роналду в Ювентусе?
AI: 7
Human: Под каким номером он играл в Манчестер Юнайтед?
Chatbot:

> Finished chain.


'7'

In [31]:
conversation_buf_mem.memory.chat_memory

InMemoryChatMessageHistory(messages=[HumanMessage(content='Под каким номером играл Криштиану Роналду в Ювентусе?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={}), HumanMessage(content='Под каким номером он играл в Манчестер Юнайтед?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})])

In [34]:
conversation_buf_mem.memory.clear()

In [36]:
# ответьте на вопросы с помощью ЛЛМки
ans = []
id_cur = df['dialogue_id'][0]
for id, q in zip(df['dialogue_id'], df['question']):
    if id != id_cur:
        conversation_buf_mem.memory.clear()
        id_cur = id
    answer = conversation_buf_mem.predict(human_input=q)
    ans.append(answer)
    # break # уберем break, когда убедимся, что работает на одном примере



> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
Answer with only one number.

Current conversation:

Human: Под каким номером играл Криштиану Роналду в Ювентусе?
AI: 7
Human: Под каким номером играл Криштиану Роналду в Ювентусе?
Chatbot:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.
Answer with only one number.

Current conversation:

Human: Под каким номером играл Криштиану Роналду в Ювентусе?
AI: 7
Human: Под каким номером играл Криштиану Роналду в Ювентусе?
AI: 7
Human: Под каким н

In [39]:
ans_int = [int(a) for a in ans]

In [40]:
df['answer'] = ans_int # запишем предсказания в датафрейм
df.to_csv('solution.csv', index=False) # сохраним решение в csv

In [41]:
df

,dialogue_id,question,answer
0,1,Под каким номером играл Криштиану Роналду в Юв...,7
1,1,Под каким номером он играл в Манчестер Юнайтед?,7
2,1,В каком году он был рожден?,1985
3,2,Сколько лет прожил величайший русский поэт Але...,37
4,2,В каком году он родился?,1799
5,2,Дуэль в каком году стала для него последней?,1837
6,3,Сколько было участников в золотом составе груп...,4
7,3,В каком году родился барабанщик золотого соста...,1940
8,3,В каком году распалась группа?,1970
9,4,В каком году родился Альберт Эйнштейн?,1879
